# Polars: Blazingly fast dataframes!! 🚀🚀 <small>by Ritchie Vink</small>

_An in memory query engine + DataFrame library written from scratch in Rust_

<div>
<img src="../img/polars_logo_black_circle.png" style="display:block;margin-left:auto;margin-right:auto;width:20%" </img>
</div>
<div>
<img src="../img/xomnia_logo.png" style="display:block;margin-left:auto;margin-right:auto;width:20%" </img>
</div>

### Goals
* Maximum runtime performance
* Composable functional API

### Topic

* Polars in a nutshell
* Expression language



## Why Polars?

1. Rustlang did not have a DataFrame library
2. Moore's law is ending
    - response: more cores.
    - numpy + pandas are single threaded and don't benefit from this hardware trend
3. Decades of database development (e.g.) query optimizations are not being used in python dataframe stack.
4. we (Python) got our ass kicked by R datatable.


## Foundations

* **Rust programming language**: memory safe programming language 
    - performance == C/C++
    - Full control over memory / close to the metal
    - Fearless concurrency
    
---

* **Apache Arrow** as memory model
    - The future of big data communication and columnar store
    - Reduction of serialization/deserialization by standardizing and optimization of columnar memory.
    - within same process, data access is free (ptr sharing)
    

<div>
<img src="../presentation_xomnia/img/rustlang.png" style="display:block;margin-left:auto;margin-right:auto;width:20%;float:left" </img>
</div>
    
<div>
<img src="../img/arrow_graph.png" style="display:block;margin-left:auto;margin-right:auto;width:40%;float:left" </img>
</div>



## Polars in a nutshell: 

* Column oriented data storage
    - No block manager -> predictable performance
* Copy on write semantics
    - "Free" clones
    - Cheap appends
* Proper missing values: missing != `NaN`
* Efficient algorithms writting from scratch w/ for max. perf.
* Query optimizations
    - Expression folding
    - Predicate pushdown
    - Projection pushdown
    - Parallelization of physical plan
* SIMD vectorization

## Polars in a nutshell: Expressions + Query optimizations

* Lazy evaluation
    - Declarative
        - e.g. describe what you want, not how to do it
    - More context for optimizations
    
---

**declarative**
> ```
> get me a cup of coffee.
> ```

**procedural**
> ```
> take 5 steps of 75cm
> turn 90 degrees
> take 3 steps of 75cm
> take 1 step of 23cm
> turn 45 degrees
> lift your hand
> ...
> ...
> lower the coffee cup on the table
> ```

The latter has less room for optimization

## Polars in a nutshell: Blazingly fast 🚀🚀

source: https://h2oai.github.io/db-benchmark/

<div>
<img src="../presentation_xomnia/img/db-benchmark.png" width=700>
</div>



## Compared to Pandas

**performance**

* 5-20x runtime improvement

**memory requirements**
* pandas: ~10x dataset size
* polars: ~2-4x dataset size


# Expressions

* `Fn(Series) -> Series`
    - expr output is input for another expr
    - Very composable expression language
    - can be combined indefinitetly
* Lazy evaluated
    - Optimizable 
* Reduces the need for custom (slow) python functions
* Embarrassingly parallel


---
    
They work differently depending on the context.

**Contexts**
* Selection
* Groupby

```python
# sum the column "foo"
df.select(pl.sum("foo"))

# sum values of "foo" grouped by "bar"
df.groupby("bar").agg(pl.sum("foo"))
```


In [2]:
import polars as pl
from polars import col, lit
import numpy as np
import pandas as pd

df = pl.DataFrame(
    {
        "A": [1, 2, 3, 4, 5],
        "fruits": ["banana", "banana", "apple", "apple", "banana"],
        "B": [5, 4, 3, 2, 1],
        "cars": ["beetle", "audi", "beetle", "beetle", "beetle"],
        "optional": [28, 300, None, 2, -30],
    }
)
df

A,fruits,B,cars,optional
i64,str,i64,str,i64
1,"""banana""",5,"""beetle""",28
2,"""banana""",4,"""audi""",300
3,"""apple""",3,"""beetle""",null
4,"""apple""",2,"""beetle""",2
5,"""banana""",1,"""beetle""",-30


## Selection context

* You can selection any value from a DataFrame.
* The data flowing through the expressions are columns in the `DataFrame`
* Every expression is evaluated in parallel.



In [3]:
# let's start simple

df.select([
    col("A"),                                      # select column "A" (syntactix sugar for `col("A")`)
    col("A").reverse().suffix("_reverse"),        # select column "A" -> reverse -> rename to "A_reverse"
])

A,A_reverse
i64,i64
1,5
2,4
3,3
4,2
5,1


In [4]:
# We select everything in normal order
# Then we select everything in reversed order
(df.select([
    pl.all(),  # we can also exclude/use regexes
    pl.all().reverse().suffix("_reverse")
]))

A,fruits,B,cars,optional,A_reverse,fruits_reverse,B_reverse,cars_reverse,optional_reverse
i64,str,i64,str,i64,i64,str,i64,str,i64
1,"""banana""",5,"""beetle""",28,5,"""banana""",1,"""beetle""",-30
2,"""banana""",4,"""audi""",300,4,"""apple""",2,"""beetle""",2
3,"""apple""",3,"""beetle""",null,3,"""apple""",3,"""beetle""",null
4,"""apple""",2,"""beetle""",2,2,"""banana""",4,"""audi""",300
5,"""banana""",1,"""beetle""",-30,1,"""banana""",5,"""beetle""",28


In [5]:
# We can also do aggregations
(df.select([
    pl.all().sum().suffix("_sum"),   # select all columns -> compute the sum -> add "_sum" to name
    pl.all().min().suffix("_min")    # select all columns -> compute the min -> add "_min" to name
]))

A_sum,fruits_sum,B_sum,cars_sum,optional_sum,A_min,fruits_min,B_min,cars_min,optional_min
i64,str,i64,str,i64,i64,str,i64,str,i64
15,null,15,null,300,1,null,1,null,-30


In [6]:
# Aggregation and normal projections can be combined
(df.select([
    col("A").rank().alias("rank_A"),   # select column "A" -> compute the rank -> rename to "rank_A"
    col("A").std().alias("A_std")      # select column "A" columns -> compute the std-dev -> rename to "A_std"
]))

rank_A,A_std
f32,f64
1,1.5811388300841898
2,1.5811388300841898
3,1.5811388300841898
4,1.5811388300841898
5,1.5811388300841898


## More on selecting columns

We can also select multiple columns by:
    
* regex: `col("^A.*$")`
* exclude columns: `pl.all().exclude("fruits")`
* dtypes: `col(pl.Utf8)`

## Filtering

In [7]:
# we can use simple filters

df.filter(col("fruits") == "banana")

A,fruits,B,cars,optional
i64,str,i64,str,i64
1,"""banana""",5,"""beetle""",28
2,"""banana""",4,"""audi""",300
5,"""banana""",1,"""beetle""",-30


In [8]:
eq_banana = col("fruits") == "banana"

(df.select([
    col("fruits").filter(eq_banana).count().alias("n_eq_banana"),
    col("A").filter(eq_banana).sum().alias("sum_A_eq_banana"),
]))

n_eq_banana,sum_A_eq_banana
u32,i64
3,8


In [9]:
# we can combine aggregations, filters, arithmetic and projection
df.select([
    "A",
    "B",
    "fruits",
    col("A").mean().alias("A_mean"),
    col("B").filter(col("A") > 2).sum().alias("sum_a_gt_1"),
    col("B").filter(col("fruits") == "banana").sum().alias("sum_a=banana"),
    ((np.exp(col("B")) ** 2 + col("A").mean()) / 2).alias('arithmetic_result')
])

A,B,fruits,A_mean,sum_a_gt_1,sum_a=banana,arithmetic_result
i64,i64,str,f64,i64,i64,f64
1,5,"""banana""",3,6,10,1.1014732897403357e4
2,4,"""banana""",3,6,10,1491.978993520864
3,3,"""apple""",3,6,10,203.21439674636756
4,2,"""apple""",3,6,10,28.79907501657212
5,1,"""banana""",3,6,10,5.194528049465324


In [10]:
# we can combine columns by a predicate

# pandas
df_pd = df.to_pandas()
df_pd["new"] = df_pd["B"]
df_pd[df_pd["fruits"] != "banana"] = -1
print(df_pd)

# polars
df.with_column(
    # similar to an if-else-expr
    pl.when(col("fruits") == "banana").then(col("B")).otherwise(-1).alias("new")
)

   A  fruits  B    cars  optional  new
0  1  banana  5  beetle      28.0    5
1  2  banana  4    audi     300.0    4
2 -1      -1 -1      -1      -1.0   -1
3 -1      -1 -1      -1      -1.0   -1
4  5  banana  1  beetle     -30.0    1


A,fruits,B,cars,optional,new
i64,str,i64,str,i64,i64
1,"""banana""",5,"""beetle""",28,5
2,"""banana""",4,"""audi""",300,4
3,"""apple""",3,"""beetle""",null,-1
4,"""apple""",2,"""beetle""",2,-1
5,"""banana""",1,"""beetle""",-30,1


In [11]:
# or we could keep combining predicates
df.with_column(
    pl.when(col("fruits") == "banana")
    .then(col("B"))
    .when(col("fruits") == "apple")
    .then(12)
    .otherwise(-1).alias("new")
)

A,fruits,B,cars,optional,new
i64,str,i64,str,i64,i64
1,"""banana""",5,"""beetle""",28,5
2,"""banana""",4,"""audi""",300,4
3,"""apple""",3,"""beetle""",null,12
4,"""apple""",2,"""beetle""",2,12
5,"""banana""",1,"""beetle""",-30,1


```python
def pandas_some_function(df: pd.DataFrame) -> pd.DataFrame:
    # often seen and needed to write pure functions
    # but its very expensive
    df = df.clone()
    df["a"] = df["b"] + df["c"]
    return df
    
# all runs sequential
(pandas_df
    .pipe(pandas_some_function)
    .pipe(pandas_some_other_function)
)

def polars_some_function() -> pl.Expr:
    return col("b") + col("c").alias("a")
    
# sequential + parallel
(polars_df
     .with_column(foo_fn)
     .with_columns([bar_fn, ham_fn])  # parallel
     .select([
         polars_some_function(),      # parallel
         polars_some_other_function
         ...
         compute_bar()
     ])
)
```

## Groupby context

* syntax: `df.groupby(..).agg([exprs..])`
* The data flowing through the expressions are the groups of the groupby operation
* Every expression is evaluated in parallel.

In [12]:
(df.groupby("fruits")
 .agg([
     col("cars")
 ])
)

fruits,cars
str,list
"""apple""","[""beetle"", ""beetle""]"
"""banana""","[""beetle"", ""audi"", ""beetle""]"


In [13]:
(df.groupby("fruits")
 .agg([
     col("cars").filter(col("cars") == "beetle"),
     col("cars").filter(col("cars") == "beetle").count().alias("beetle_count"),
 ])
)

fruits,cars,beetle_count
str,list,u32
"""apple""","[""beetle"", ""beetle""]",2
"""banana""","[""beetle"", ""beetle""]",2


In [14]:
# Example of multiple aggregations in pandas
# however combining aggregations seems not possible? 

# df_pd = df.to_pandas()
# df_pd.groupby("fruits").agg({
#     "A": ["shift", "sum"]
# })

(df.groupby("fruits")
    .agg([
        (col("A").reverse().rolling_min(window_size=2) ** 2),
        (col("A").reverse().rolling_min(window_size=2) ** 2).sum(),
    ])
)

fruits,A,A_sum
str,list,f64
"""apple""","[null, 9]",9
"""banana""","[null, 4, 1]",5


# Window functions!

* Expression with superpowers.
* Aggregation in selection context
* Groupby over different columns


```python
col("foo").aggregation_expression(..).over("column_used_to_group")

col("foo").multiple_agg_exprs(..).over("column_used_to_group")
```

In [15]:
(df.sort("fruits")
 .select([
    "fruits",
    "cars",
     "A",
    col("A").apply(lambda x: x * 2).alias("a_new"),
    col("A").list().over("fruits").alias("group_lists_by_fruits"),
    col("A").mean().over("fruits").alias("mean_by_cars"),
    col("A").rank().over("fruits").flatten().alias("A_ranked_by_fruits"),
]))

fruits,cars,A,a_new,group_lists_by_fruits,mean_by_cars,A_ranked_by_fruits
str,str,i64,i64,list,f64,f32
"""apple""","""beetle""",3,6,"[3, 4]",3.5,1
"""apple""","""beetle""",4,8,"[3, 4]",3.5,2
"""banana""","""beetle""",1,2,"[1, 2, 5]",2.6666666666666665,1
"""banana""","""audi""",2,4,"[1, 2, 5]",2.6666666666666665,2
"""banana""","""beetle""",5,10,"[1, 2, 5]",2.6666666666666665,3


## Final words

* Full lazy execution -> more optimizations
* Feature complete.
* Possible backend for more tools/runtimes
    - WASM
    - anything that could bind with C
    - nu-shell
    - Elixir
    - R bindings are being discussed atm
* Shoot!